# Support Vector Machines (SVM)
SVM used for __classification__, __regression__ and __outliers detection__.

__Pros:__
- Hiệu quả với dữ liệu nhiều chiều
- Hiệu quả với dữ liệu có n_features > n_sample
- Chỉ cần 1 subset để training--> hiệu quả về mặt lưu trữ ram
- Sử dụng kết hợp với __Kernel__ function để tuỳ chỉnh cách fitting

__Cons:__
- Nếu n_feature >> n_sample, cần phải tunning thật kỹ kernel và regularization term để tránh overfitting
- SVMs không cung cấp trực tiếp probability estimates prediction, mà cần tính toán qua 5-fold CV.

## SVM Classification (SVC)
Sử dụng cho binary hoặc multi-class classification, tìm 1 hyper-plane  hoặc 1 set hyper-planes trong không gian đa chiều, mà tại đó khoảng cách từ hyper-plane tới điểm gần nhất của mỗi class là lớn nhất
- `SVC` và `NuSVC` khá giống nhau, chỉ khác ở bộ tham số và hàm tính toán
- `LinearSVC` chạy nhanh do thực hiện linear kernel, do đó không thể chỉnh `kernel` parameter trong `LinearSVC`

**Multi-class classification**
- `SVC` và `NuSVC` thực hiện “one-versus-one” approach cho multi-class classification, có tổng cộng là `n_classes` * (`n_classes` - 1) / 2 sub-model
- `LinearSVC` thực hiện "one-vs-the-rest", cho nên training chỉ có `n_classes` sub-model (preferer to use)

**Unbalanced problems**
- Sử dụng `class_weight` (chỉ có trong SVC, not NuSVC) hoặc `sample_weight` parameters để thay đổi weight cho các minority class 

### SVC
- Thực hiện dựa trên pphương pháp 'libsvm'
- Phù hợp với dữ liệu vừa và nhỏ (For large datasets consider using LinearSVC or SGDClassifier instead)
- Hỗ trợ sử dụng kernel function, tunning qua tham số:
    - gamma
    - coef0
    - degree
- Với multi-class, sử dụng pp tiếp cận one-vs-one

In [114]:
from sklearn.svm import SVC

svc = SVC(
    C=1.0, # mức độ penalty, C càng lớn thì phạt càng ít (ngược với alpha của L2, L1)
    kernel='rbf', # hàm kernel được sử dụng để transform data {‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}
    degree=3, # bậc của 'poly' kernel
    gamma='scale', # hệ số của hàm kernel ‘rbf’, ‘poly’ and ‘sigmoid’.
    coef0=0.0, # hệ số independent term của hàm kernel ‘poly’ and ‘sigmoid’.
    shrinking=True, 
    probability=False, # có ước lượng xác suất của prediction hay ko ?
    tol=0.001, # ngưỡng của điều kiện dừng
    cache_size=200,
    class_weight=None,
    verbose=False,
    max_iter=-1,
    decision_function_shape='ovr',
    break_ties=False,
    random_state=None,
)

svc.fit(X_train, y_train)
svc.score(X_test, y_test)

0.898

### LinearSVC
- Thực hiện tương tự như SVC kernel = 'linear' và dựa trên phương pháp 'liblinear'
- Phù hợp với dữ liệu lớn
- Hỗ trợ sử dụng regularization {L1, L2}
- Với multi-class, pp tiếp cận là one-vs-rest

In [115]:
from sklearn.svm import LinearSVC

lsvc = LinearSVC(
    penalty='l2',
    loss='squared_hinge',
    dual=True,
    tol=0.0001,
    C=1.0,
    multi_class='ovr',
    fit_intercept=True,
    intercept_scaling=1,
    class_weight=None,
    verbose=0,
    random_state=None,
    max_iter=1000,
)

lsvc.fit(X_train, y_train)
lsvc.score(X_test, y_test)

/Users/khongdat/miniforge3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.74

## SVM Regression (SVR)
tương tự SVC

In [116]:
# SVR
from sklearn.svm import SVR

# linearSVR
from sklearn.svm import LinearSVR

## OneClassSVM Outlier Detection
- Unsupervised Outlier Detection, using 'libsvm'

In [119]:
from sklearn.svm import OneClassSVM

one = OneClassSVM(
    kernel='rbf',
    degree=3,
    gamma='scale',
    coef0=0.0,
    tol=0.001,
    nu=0.5,
    shrinking=True,
    cache_size=200,
    verbose=False,
    max_iter=-1,
)

one.fit(X_train)


OneClassSVM()

## Kernel Function

[Chi tiết 1.4.6. Kernel functions](https://scikit-learn.org/stable/modules/svm.html#kernel-functions)

**idea:** Sử dụng 1 hàm phi tuyến tính (gọi là `kernel`) để biến đổi data ban đầu không phân biệt được linear thành data với chiều không gian mới và sử dụng linear-line hoặc 1 mặt phẳng để phân tách data (đã biến đổi) và vẫn trên margin lớn nhất
![image.png](_images/4_ML_Algorithms/classification/kernel_f.png)
```python
kernel = 'linear', 'poly', 'rbf', 'sigmoid', 'precomputed',...
```
![image.png](_images/4_ML_Algorithms/classification/kernel_t.png)

**ưu điểm**
- thực hiện tốt trên nhiều loại datasets
- Linh động bởi có thể thực hiện bằng nhiều hàm kernel khác nhau
- Thực hiện tốt trên cả data nhiều chiều hay ít chiều

**nhược điểm**
- Thời gian chạy lâu và tốn nhiều dung lượng khi chạy sample size lớn (lớn hơn 50000 samples)
- Cần cần thận trong việc normalize data
- Không cung cấp xác suất ước lượng của phân phối (tuy nhiên có thể tính được bằng `Platt scaling`)
- Khó thể giải thích việc dự đoán của model, diễn giải

**RBF kernel**

- Hàm `RBF kernel` sẽ sử dụng khoảng cách giữa các obs để phân tách, dựa trên tham số `gammar`
- `Gammar` thể hiện độ nhọn của phân phối của class "y<>1", khi `gammar` càng lớn thì càng ít vùng xung quanh obs y<>1 được dự báo là y<>1 (hay xác suất của phân phối y<>1 sẽ nhọn hơn, các điểm ở xa obs y<>1 sẽ có xác suất để y<>1 ít hơn)
- `Gammar` tăng thì model càng bị over-fitting

**Poly kernel**

- Hàm `poly kernel` sẽ sử dụng hàm bậc `degree` để biến đổi data và phân tách dựa trên data đã biến đổi
- `degree` càng lớn thì model càng complex, dễ bị over-fitting